# Lab 2 - Exploring Text Data

### Eric Smith and Jake Carlson

## Introduction
For this lab we will be examining questions and answers on the popular programming Q&A website, Stack Overflow. Stack Overflow allows programmers who are stuck on a problem ask the community about ways to resolve or circumvent said problem. This helps people write more accurate code faster. However, the community can be pretty hard on those who do not fully understand their question or are disrespectful. It could also be the case that the few developers who have experience with a problem similar to yours are not on the website when you post your question. If too much time passes, your post may be burried forever. This makes for some interesing questions. When should you make your post so that it has the highest chance of being answered? Are there specific keywords or phasing that encourage other programmers to answer your questions? Are there specific languages that have their questions answered faster than others? These are questions we will be exploring int his lab.

## Business Understanding

### Motivations
Stack Overflow is a go-to resource for developers. Questions are often distilled down to a single block of code that is easily digestible, and can be matched to the question you came to the site for. However, if you're truly stuck on something, it can seem like an eternity of waiting before someone responds to your question, and sometimes, it may not be answered at all. If you're working in industry and spending a lot of time waiting for a question to be answered, you could fall behind schedule and miss deadlines. These delays could cost your company a great deal in lost revenue and tech debt. Therefore, accurately predicting the amount of time to get a question answered can be seen as a valuable project management tool.

### Objectives
It would be useful if you could predict how quickly your question will be answered if you post it at various times through out the day. For a prediction tool like this to be useful, we want something that is 80-95% accurate in predicting when a question will be answered by. As a project manager, you want your developers to be asking questions that are concise and respectful. Therefore, a tool that could recommend keywords to add to your post to make it more attractive, and provide you with approximate gains in time for reformatting, would also be a valuable project management tool.

## Data Understanding

### Data Attributes
The following is a list of attributes in the data, their data types, and a brief description of the attribute.

#### Questions
- **Id** (nominal): A unique identifier for each question
- **OwnerUserId** (nominal): A unique identifier for the person who posted the question
- **CreationDate** (ordinal): A timestamp of when the question was posted
- **ClosedDare** (ordinal): A timestamp of when the question was closed, if the question wasn't closed this field is NaN
- **Score** (ordinal): ratio?? The number of upvotes a post has
- **Title** (text): A title for the question
- **Body** (text): The question body

#### Answers
- **Id** (nominal): A unique identifier for each answer
- **OwnerUserId** (nominal): A unique identifier for the person who posted the answer
- **CreationDate** (ordinal): A timestamp of when the answer was posted
- **ParentId** (nominal): The unique identifier of the question this answer is for
- **Score** (ordinal): ratio?? The number of upvotes an answer has
- **Body** (text): The answer body


## Data Quality
Our data it too big, so we will take a rendom subsample of 40% of the original data set.

In [1]:
import numpy as np
import pandas as pd

In [2]:
# read data
df = pd.read_csv('./data/Questions.csv', encoding='ISO-8859-1')

In [3]:
df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body
0,80,26.0,2008-08-01T13:57:07Z,NaN,26,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...
1,90,58.0,2008-08-01T14:41:24Z,2012-12-26T03:45:49Z,144,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...
2,120,83.0,2008-08-01T15:50:08Z,NaN,21,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...
3,180,2089740.0,2008-08-01T18:42:19Z,NaN,53,Function for creating color wheels,<p>This is something I've pseudo-solved many t...
4,260,91.0,2008-08-01T23:22:08Z,NaN,49,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1264216 entries, 0 to 1264215
Data columns (total 7 columns):
Id              1264216 non-null int64
OwnerUserId     1249762 non-null float64
CreationDate    1264216 non-null object
ClosedDate      55959 non-null object
Score           1264216 non-null int64
Title           1264216 non-null object
Body            1264216 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 67.5+ MB


Trial and error has shown us that 1,264,000 elements is too many to work with. Therefore, we will use a random sample to reduce our data set size to 12,000 elements.

In [5]:
df = df.sample(n=12000, replace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 1212819 to 323450
Data columns (total 7 columns):
Id              12000 non-null int64
OwnerUserId     11862 non-null float64
CreationDate    12000 non-null object
ClosedDate      537 non-null object
Score           12000 non-null int64
Title           12000 non-null object
Body            12000 non-null object
dtypes: float64(1), int64(2), object(4)
memory usage: 750.0+ KB


The body of each question is formatted in HTML, so we will need to parse through each body and remove the HTML tags. We also need to replace HTML entities with their character representations. Finally, many questions have accompanying code blocks, so we will parse out the code blocks into a new attribute for each entry.

In [6]:
from bs4 import BeautifulSoup
import html
import re

# compile regexes
tag_re = re.compile('<[^<]+?>')
newline_re = re.compile('\n+')

def clean_html(body_text):
    soup = BeautifulSoup(body_text, 'html.parser')
    code_tags = soup.findAll('code')
    code_text = ""
    
    # unescape html entities
    body_text = html.unescape(body_text)
    
    # remove code blocks, saving blocks to new string
    for c in code_tags:
        body_text = body_text.replace("<code>{}</code>".format(c.string),
                                      '')
        if c.string:
            code_text += c.string

    # remove remaining tags and multiple newlines
    body_text = tag_re.sub('', body_text)
    body_text = newline_re.sub('\n', body_text)
    
    return body_text, code_text

as_list = df.Body.tolist()
body_list = []
code_list = []
for i in range(len(as_list)):
    body_text, code_text = clean_html(as_list[i])
    body_list.append(body_text)
    code_list.append(code_text)

# update body column
df = df.assign(Body=body_list)
# add code column
df = df.assign(Code=code_list)
df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Code
1212819,38840540,4517895.0,2016-08-09T01:06:16Z,NaN,0,Using USYNCHRONIZED persistence context with H...,I'm trying to migrate application which uses u...,@PersistenceContext(type = PersistenceContextT...
117826,4812460,150062.0,2011-01-27T03:32:53Z,NaN,2,Combine three select queries?,I would like to combine these three sql querie...,"SELECT homeupload, COUNT(*) FROM link GROUP BY..."
184545,7101540,45580.0,2011-08-18T01:50:03Z,NaN,0,Attach sameplerate or dmix ALSA plugin to OSS ...,I have an application in an embedded system th...,
935218,31152890,5068035.0,2015-07-01T05:04:02Z,NaN,1,Chrome Browser is not opening for running test...,"I have setup of php zendframework , selenium. ...",/usr/bin/google-chrome --disable-background-ne...
1197793,38442270,3822621.0,2016-07-18T16:59:29Z,NaN,0,Grep/Sed/Awk Trim text around IP address in sp...,I've been struggling with this issue for quite...,Built inbound ICMP connection for faddr [hostn...


With the HTML tags removed and the code isolated, the question body is much cleaner. Now lets add another column for the time in minutes it took to get the question answered. For this value, we will look for the highest rated answer for each question and use the time when that answer was posted.

In [7]:
df_ans = pd.read_csv('./data/Answers.csv', encoding='ISO-8859-1')
df_ans.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body
0,92,61.0,2008-08-01T14:45:37Z,90,13,"<p><a href=""http://svnbook.red-bean.com/"">Vers..."
1,124,26.0,2008-08-01T16:09:47Z,80,12,<p>I wound up using this. It is a kind of a ha...
2,199,50.0,2008-08-01T19:36:46Z,180,1,<p>I've read somewhere the human eye can't dis...
3,269,91.0,2008-08-01T23:49:57Z,260,4,"<p>Yes, I thought about that, but I soon figur..."
4,307,49.0,2008-08-02T01:49:46Z,260,28,"<p><a href=""http://www.codeproject.com/Article..."


In [8]:
df_ans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014516 entries, 0 to 2014515
Data columns (total 6 columns):
Id              int64
OwnerUserId     float64
CreationDate    object
ParentId        int64
Score           int64
Body            object
dtypes: float64(1), int64(3), object(2)
memory usage: 92.2+ MB


In [9]:
ids = df.Id.unique()
df_ans = df_ans[df_ans.ParentId.isin(ids)]
df_ans.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18976 entries, 290 to 2014453
Data columns (total 6 columns):
Id              18976 non-null int64
OwnerUserId     18862 non-null float64
CreationDate    18976 non-null object
ParentId        18976 non-null int64
Score           18976 non-null int64
Body            18976 non-null object
dtypes: float64(1), int64(3), object(2)
memory usage: 1.0+ MB


We will clean the answers data frame using the same function as the questions data frame.

In [10]:
as_list = df_ans.Body.tolist()
body_list = []
code_list = []
for i in range(len(as_list)):
    body_text, code_text = clean_html(as_list[i])
    body_list.append(body_text)
    code_list.append(code_text)

# update body column
df_ans = df_ans.assign(Body=body_list)
# add code column
df_ans = df_ans.assign(Code=code_list)
df_ans.head()

,Id,OwnerUserId,CreationDate,ParentId,Score,Body,Code
290,8811,194.0,2008-08-12T13:20:55Z,8800,127,There is an actual Data Type called KeyValuePa...,"KeyValuePair<string, string> myKeyValuePair = ..."
291,8813,267.0,2008-08-12T13:22:01Z,8800,1,"Dictionary Class is exactly what you want, cor...",
292,8814,42.0,2008-08-12T13:23:27Z,8800,5,There is a KeyValuePair built-in type. As a ma...,
293,8816,372.0,2008-08-12T13:25:12Z,8800,12,One possible thing you could do is use the Dic...,"public class TokenTree : Dictionary<string, st..."
294,8818,267.0,2008-08-12T13:25:56Z,8800,2,@Jay Mooney: A generic Dictionary class in .NE...,foreach(object key in h.keys)\n{\n string ...


In [11]:
# adapted from https://stackoverflow.com/questions/2788871/date-difference-in-minutes-in-python
from datetime import datetime
import time

fmt = '%Y-%m-%dT%H:%M:%SZ'
def get_minutes_diff(timestamp1, timestamp2):
#     if np.isnan(x.ClosedDate):
#         return np.nan
    d1 = datetime.strptime(timestamp1, fmt)
    d2 = datetime.strptime(timestamp2, fmt)
    
    # convert to unix timestamp
    d1_ts = time.mktime(d1.timetuple())
    d2_ts = time.mktime(d2.timetuple())
    
    # convert to minutes
    return int(d2_ts - d1_ts) / 60

durrations = []
for i, row in df.iterrows():
    answer = df_ans[ df_ans.ParentId == row.Id ]
    answer = answer[ answer.Score == answer.Score.max() ]
    answer_time = answer.CreationDate.values
    if len(answer_time) > 0:
        answer_time = answer_time[0]
        df.loc[i, 'Durration'] = get_minutes_diff(row.CreationDate, 
                                                  answer_time)

df.head()

,Id,OwnerUserId,CreationDate,ClosedDate,Score,Title,Body,Code,Durration
1212819,38840540,4517895.0,2016-08-09T01:06:16Z,NaN,0,Using USYNCHRONIZED persistence context with H...,I'm trying to migrate application which uses u...,@PersistenceContext(type = PersistenceContextT...,1205.300000
117826,4812460,150062.0,2011-01-27T03:32:53Z,NaN,2,Combine three select queries?,I would like to combine these three sql querie...,"SELECT homeupload, COUNT(*) FROM link GROUP BY...",4.933333
184545,7101540,45580.0,2011-08-18T01:50:03Z,NaN,0,Attach sameplerate or dmix ALSA plugin to OSS ...,I have an application in an embedded system th...,,338.433333
935218,31152890,5068035.0,2015-07-01T05:04:02Z,NaN,1,Chrome Browser is not opening for running test...,"I have setup of php zendframework , selenium. ...",/usr/bin/google-chrome --disable-background-ne...,141.150000
1197793,38442270,3822621.0,2016-07-18T16:59:29Z,NaN,0,Grep/Sed/Awk Trim text around IP address in sp...,I've been struggling with this issue for quite...,Built inbound ICMP connection for faddr [hostn...,88.283333


Save the cleaned data to a persistance file.

In [13]:
df.to_csv('./clean-data/Questions.csv', sep=',')
df_ans.to_csv('./clean-data/Answers.csv', sep=',')

## Data Visualization

In [14]:
df = pd.read_csv('./clean-data/Questions.csv')
df_ans = pd.read_csv('./clean-data/Answers.csv')

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english')
bag_words = count_vect.fit_transform(df.Title.tolist())

In [7]:
len(count_vect.vocabulary_)

10158

In [8]:
df_title = pd.DataFrame(data=bag_words.toarray(), 
                        columns=count_vect.get_feature_names())

In [9]:
# print 10 most common words
df_title.sum().sort_values()[-10:]

function    318
value       323
php         356
java        377
jquery      379
error       475
data        476
android     479
file        529
using       890
dtype: int64